# Userguide: Fatalities Categories (fatcat)

This notebook presents 


This notebook creates a way to observe countries' levels of violence and their likelihood of transitioning to various levels of violence over time periods. 

This notebook will create a new categorical variable which converts the actuals and predictions into the following groups: (0) Non-violent; (1) Low-violence; (2) Violent; (3) High-violence; (4) Extreme Violence.



### Configuration and Packages

In [ ]:
# Basics
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
import warnings
warnings.filterwarnings('ignore')

# Views 3
from viewser.operations import fetch
from viewser import Queryset, Column
import views_runs
from views_partitioning import data_partitioner, legacy
from stepshift import views
from views_runs import storage, ModelMetadata
from views_runs.storage import store, retrieve, fetch_metadata
from views_forecasts.extensions import *


# Packages from Mapper2
from ingester3.ViewsMonth import ViewsMonth
from ingester3.Country import Country
from datetime import date


# Packages from this repository, Tools folder
import sys
sys.path.append('../')
sys.path.append('../Tools')
sys.path.append('../Intermediates')
sys.path.append('../SystemUpdates')

import os
from pathlib import Path

from fatcat import get_fatalities, get_fatcat, get_fatcat_list, clean_list, get_escalation_list 
from fatcat import get_deescalation_list, get_nochange_list, get_escalation_shortlist, get_deescalation_shortlist, get_nochange_shortlist
from fatcat import fatcat_compare, plot_pastfuture, plot_pastfuture_log, plot_fatcat, vid2date



In [ ]:
# Fat001
#dev_id = 'Fatalities001'
#run_id = dev_id 
#EndOfHistory = 517
#prod_id = '2023_00_t01'
#level = 'cm'

#Fat002
dev_id = 'Fatalities002'
run_id = dev_id 
EndOfHistory = 509
prod_id = '2022_04_t01'
level = 'cm'

import os
home = os.path.expanduser("~")
Mydropbox = home + '/Dropbox (ViEWS)/ViEWS_new/'
Monthly_updates = Mydropbox + 'DataReleases/MonthlyUpdates/'
localgitpath = home + '/Desktop/VIEWS_new/'


##################
print('OS Login is:', os.getlogin())
print('OS path is set to:', home)
print('Local Git path is set to:', localgitpath)
print('Dropbox path is set to:', Mydropbox)

In [ ]:
# Print month_ids for the chosen period
print(ViewsMonth.from_year_month(year=2022, month=5)) 

# 1: Fetching Past and Future Fatalities

The following calls functions from 'fatcat.py' to create a df with all fatalities (actuals) and predictions (future). 

    1: Pull in the actuals, restrict the data to (EndOfHistory-40) to EndOfHistory

    2: Pull in the predictions, steps EndOfHistory+1 to EndOfHistory+36 months into the future
    
    3: Merge the DFs: create a single df with actuals and predictions (fatalities)

## get_fatalities

This function creates a DataFrame with the number of fatalities for each country-month from 36 months before and after EndOfHistory. The function reads in the actuals using *Queryset* and predictions using *read_store*, cleans the data, and combines their content into a single DataFrame. The function uses the parameters set in the 'configuration cell' at the top of the document. These parameters include: 

    1. Level = The VIEWS level of observation (e.g. cm, pgm, am)
    2. Dev_id = The VIEWS development ID (e.g. Fatalities002)
    3. EndOfHistory = The EndOfHistory of interest (e.g. EndOfHistory = 509)

In [ ]:
help(get_fatalities)

In [ ]:
fatalities = get_fatalities(level=level, dev_id=dev_id, EndOfHistory=EndOfHistory)

In [ ]:
fatalities

# 2: Creating Fatalities Categories (fatcat)


## get_fatcat

The 'get_fatcat' function populates the DataFrame created by 'get_fatalities' with information that best categorizes the status of each country-month based on their respective fatalities counts. The 'rolling_months_prior' parameter allows users to define their relevant number of month prior for violence in a given country.    
<br>
Example: 
    
    df = get_fatcat(fatalities, rolling_months_prior=4)

**Fatcat** presents a categorical variable ranging from 0 to 4. These are: 

		0: Peaceful = a country-month without any fatalities
		1: Low-violence = a country-month with 1-9 fatalities
	    2: Violent = a country-month with more than 9-99 fatalities
	    3: High-violence = a country-month with more than 100-999 fatalities
	    4: Extreme violence = a country-month with more than 1000 fatalites
        
**Fatcat_change** reports the change in fatcat from one month to the next for a given country-month. These changes range from: 

		-4: Extreme De-escalation = When a country changes from extreme violence to non-violent in one month
		-3: Rapid De-escalation = When a country moves from high violence to non-violent in one month
	    -2: De-Escalation = When a country moves from high violent to non-violent in one month
	    -1: Low De-esclation = When a country moves from low violence to non-violent in one month
	     0: Staus Quo = When a country does not observe a change in violence level form month to month
	     1: Low Escalation = When a country moves from non-violent low violence to in one month. 
	     2: Escalation = When a country moves from non-violent to violent in one month
	     3: Rapid Escalation = When a country moves from non-violent to high violent in one month
	     4: Extreme Escalation = When a country moves from non-violent to high violence in one month

**Fatcat_nominal** reports the nominal category name for each country-month. These range from: 

		 1: Non-violent = a country-month with 0 fatalities
         2: Low Violence = a country-month with 1-9 fatalities
         3: Violence = a country-month with more than 9-99 fatalities
         4: High-violence = a country-month with more than 100-999 fatalities
         5: Extreme Violence = a country-month with more than 1000 fatalites
         
**Monthly_change** reports the nominal category change for each country-month's 'fatcat_change'. These range from: 

		-4: Extreme De-escalation = When a country changes from extreme violence to non-violent in one month
		-3: Rapid De-escalation = When a country moves from high violence to non-violent in one month
	    -2: De-Escalation = When a country moves from high violent to non-violent in one month
	    -1: Low De-esclation = When a country moves from low violence to non-violent in one month
	     0: Staus Quo = When a country does not observe a change in violence level form month to month
	     1: Low Escalation = When a country moves from non-violent low violence to in one month. 
	     2: Escalation = When a country moves from non-violent to violent in one month
	     3: Rapid Escalation = When a country moves from non-violent to high violent in one month
	     4: Extreme Escalation = When a country moves from non-violent to high violence in one month

**Rolling_mean_violence** the mean level of violence over the user-defined 'rolling_months_prior'. This variable provides the user with general information on the level of violence in the country over its recent past.

**Intensity_lvl** categorizes the rolling_mean_violence for each country-month into the following categories:

		 0: Peacefull = The country has observed 0 mean fatalities over the 'rolling_months_prior'
		 1: Violent = The country has observed a less than 100 mean fatalities over the 'rolling_months_prior'
	     2: High-Violence = The country has observed a over 100 mean fatalities over the 'rolling_months_prior'



In [ ]:
fatcat = get_fatcat(fatalities, rolling_months_prior=3)

In [ ]:
fatcat

## fatcat_compare

The 'fatcat_compare' function populates a DataFrame with fatalities data for each country at two user-defined step parameters. 
<br>
<br>Example use: 
    
    df = fatcat_compare(fatcat, step_value=0, step_value2=3)


**fatalities_step_x** reports the number of fatalities at the user defined country-step

**fatcat_step_x** presents a categorical variable ranging from 0 to 4 for the country-step These are: 

		0: Peaceful = a country-month without any fatalities
		1: Low-violence = a country-month with 1-9 fatalities
	    2: Violent = a country-month with more than 9-99 fatalities
	    3: High-violence = a country-month with more than 100-999 fatalities
	    4: Extreme violence = a country-month with more than 1000 fatalites

        
**Fatcat_change_step_x** reports the change in fatcat from step_x at (t+1) to step_x at (t). These changes range from: 

		-4: Extreme De-escalation = When a country changes from extreme violence to non-violent in one month
		-3: Rapid De-escalation = When a country moves from high violence to non-violent in one month
	    -2: De-Escalation = When a country moves from high violent to non-violent in one month
	    -1: Low De-esclation = When a country moves from low violence to non-violent in one month
	     0: Staus Quo = When a country does not observe a change in violence level form month to month
	     1: Low Escalation = When a country moves from non-violent low violence to in one month. 
	     2: Escalation = When a country moves from non-violent to violent in one month
	     3: Rapid Escalation = When a country moves from non-violent to high violent in one month
	     4: Extreme Escalation = When a country moves from non-violent to high violence in one month
         
**Intensity_lvl_step_x** categorizes the rolling_mean_violence_step_x for the country-step

		 0: Peacefull = The country has observed 0 mean fatalities over the 'rolling_months_prior'
		 1: Violent = The country has observed a less than 100 mean fatalities over the 'rolling_months_prior'
	     2: High-Violence = The country has observed a over 100 mean fatalities over the 'rolling_months_prior'

**Rolling_mean_violence_step_x** the mean level of violence over the user-defined 'rolling_months_prior'. This variable provides the user with general information on the level of violence in the country over its recent past.

**fatalities_step_y** reports the number of fatalities at the user defined country-step      


**fatcat_step_y** presents a categorical variable ranging from 0 to 4 for the country-step These are: 

		0: Peaceful = a country-month without any fatalities
		1: Low-violence = a country-month with 1-9 fatalities
	    2: Violent = a country-month with more than 9-99 fatalities
	    3: High-violence = a country-month with more than 100-999 fatalities
	    4: Extreme violence = a country-month with more than 1000 fatalites
        
**Fatcat_change_step_y** reports the change in fatcat from step_y at (t+1) to step_y at (t). These changes range from: 

		-4: Extreme De-escalation = When a country changes from extreme violence to non-violent in one month
		-3: Rapid De-escalation = When a country moves from high violence to non-violent in one month
	    -2: De-Escalation = When a country moves from high violent to non-violent in one month
	    -1: Low De-esclation = When a country moves from low violence to non-violent in one month
	     0: Staus Quo = When a country does not observe a change in violence level form month to month
	     1: Low Escalation = When a country moves from non-violent low violence to in one month. 
	     2: Escalation = When a country moves from non-violent to violent in one month
	     3: Rapid Escalation = When a country moves from non-violent to high violent in one month
	     4: Extreme Escalation = When a country moves from non-violent to high violence in one month
        
**Intensity_lvl_step_y** categorizes the rolling_mean_violence_step_y for the country-step

		 0: Peacefull = The country has observed 0 mean fatalities over the 'rolling_months_prior'
		 1: Violent = The country has observed a less than 100 mean fatalities over the 'rolling_months_prior'
	     2: High-Violence = The country has observed a over 100 mean fatalities over the 'rolling_months_prior'        
**Rolling_mean_violence_step_y** the mean level of violence over the user-defined 'rolling_months_prior'. This variable provides the user with general information on the level of violence in the country over its recent past.     


**Fatcat_change** Reports the change in fatcat from *step_x* to *step_y*         


**Fatcat_change_nominal** reports the nominal category for the 'fatcat_change'. These range from: 

		-4: Extreme De-escalation = When a country changes from extreme violence to non-violent in one month
		-3: Rapid De-escalation = When a country moves from high violence to non-violent in one month
	    -2: De-Escalation = When a country moves from high violent to non-violent in one month
	    -1: Low De-esclation = When a country moves from low violence to non-violent in one month
	     0: Staus Quo = When a country does not observe a change in violence level form month to month
	     1: Low Escalation = When a country moves from non-violent low violence to in one month. 
	     2: Escalation = When a country moves from non-violent to violent in one month
	     3: Rapid Escalation = When a country moves from non-violent to high violent in one month
	     4: Extreme Escalation = When a country moves from non-violent to high violence in one month

**Fatcat_step_x_nominal** reports the nominal category for the 'fatcat_step_x'. These range from: 

		0: Peaceful = a country-month without any fatalities
		1: Low-violence = a country-month with 1-9 fatalities
	    2: Violent = a country-month with more than 9-99 fatalities
	    3: High-violence = a country-month with more than 100-999 fatalities
	    4: Extreme violence = a country-month with more than 1000 fatalites
        
**Fatcat_step_y_nominal** reports the nominal category for the 'fatcat_step_y'. These range from: 

		0: Peaceful = a country-month without any fatalities
		1: Low-violence = a country-month with 1-9 fatalities
	    2: Violent = a country-month with more than 9-99 fatalities
	    3: High-violence = a country-month with more than 100-999 fatalities
	    4: Extreme violence = a country-month with more than 1000 fatalites



In [ ]:
fatcat_list = fatcat_compare(fatcat, step_value=0, step_value2=12)

In [ ]:
fatcat_list

# 3: Creating Change DataFrames

The following cells shows the countries that are expected to shift between fatcats.

## get_escalation_list

The 'get_escalation_list' takes the DataFrame populated by 'fatcat_compare' and creates a DataFrame with the countries with fatcats there are predicted to escalate from 'step_x' to 'step_y'. It's content is formated similar to 'fatcat_compare'. 

<br>
<br>Example use: 
    
    escalationlist = get_escalation_list(fatcat_list, step_value2=3)

## get_deescalation_list

The 'get_deescalation_list' takes the DataFrame populated by 'fatcat_compare' and creates a DataFrame with the countries with fatcats there are predicted to deescalate from 'step_x' to 'step_y'. It's content is formated similar to 'fatcat_compare'. 

<br>
<br>Example use: 
    
    deescalationlist = get_deescalation_list(fatcat_list, step_value2=3)

## get_nochange_list

The 'get_nochange_list' takes the DataFrame populated by 'fatcat_compare' and creates a DataFrame with the countries that are predicted to remain in their fatcat from 'step_x' to 'step_y'. It's content is formated similar to 'fatcat_compare'. 

<br>
<br>Example use: 
    
    nochangelist = get_nochange_list(fatcat_list, step_value2=3)

In [ ]:
escalationlist = get_escalation_list(fatcat_list, step_value2=12)
deescalationlist = get_deescalation_list(fatcat_list, step_value2=12)
nochangelist = get_nochange_list(fatcat_list, step_value2=12)

# 4: Creating Condensed List

The following cells shows the countries that are expected to shift between fatcats in a condensed format

## get_escalation_shortlist

The 'get_escalation_shortlist' condenses the DataFrame produced by 'get_escalation_list' to only include the country names, fatalities at step_x,fatalities at step_y, and the nominal change across these.

<br>
<br>Example use: 
    
    escalatio_short = get_escalation_shortlist(escalationlist, x, y)

## get_deescalation_shortlist

The 'get_deescalation_shortlist' condenses the DataFrame produced by 'get_deescalation_list' to only include the country names, fatalities at step_x,fatalities at step_y, and the nominal change across these.

<br>
<br>Example use: 
    
    deescalation_short = get_deescalation_shortlist(deescalationlist, x, y)

## get_nochange_shortlist

The 'get_nochange_shortlist' condenses the DataFrame produced by 'get_nochange_list' to only include the country names, fatalities at step_x,fatalities at step_y, and the nominal change across these.

<br>
<br>Example use: 
    
    nochange_short = get_nochange_shortlist(nochangelist, x, y)

In [ ]:
esc_short = get_escalation_shortlist(escalationlist, 0, 12)
deesc_short = get_deescalation_shortlist(deescalationlist, 0, 12)
nochange_short = get_nochange_shortlist(nochangelist, 0,12)

In [ ]:
display(deesc_short)

# plot_pastfuture_log

Examples: yemen (124), Ukraine (117), Syria (220), Niger (78), Nigeria (79), 

In [ ]:
plot_pastfuture_log(fatalities,17)

# plot_fatcat

This command makes line plots all fatcats for the countries and their fatalities.  

In [ ]:
plot_fatcat(fatcat,117)

# Plot alllll

This plot will plot all of the FatCat data on a single line plot :o

In [ ]:
from matplotlib.ticker import FuncFormatter

fatcat_all = fatcat.copy()
fatcat_all = fatcat_all.reset_index()
fatcat_all['fatalities_log1p'] = np.log1p(fatcat_all['fatalities'])

fig, ax = plt.subplots(figsize=(18, 3))

for country_id in fatcat_all['country_id'].unique():
    country_df = fatcat_all[fatcat_all['country_id'] == country_id]
    country_name = country_df['country'].iloc[0]
    
    # Set a custom formatter for the y-axis labels
    ax.yaxis.set_major_formatter(FuncFormatter(lambda x, pos: f'{np.round(np.expm1(x), 0):,.0f}'))
    ax.plot(country_df.loc[country_df['step'] <= 0, 'step'], country_df.loc[country_df['step'] <= 0, 'fatalities_log1p'], label=country_name + ' Actuals')
    ax.plot(country_df.loc[country_df['step'] >= 0, 'step'], country_df.loc[country_df['step'] >= 0, 'fatalities_log1p'], label=country_name + ' Predictions')

ax.axvline(x=0, color='lightcoral', linestyle='--')

ax.xaxis.grid(True, which="both", color="#cccccc", alpha=0.3, lw=0.9)
ax.yaxis.grid(True, which="both", color="#cccccc", alpha=0.3, lw=0.9)

ax.text(0, 1.13, 'Number of Reported and Predicted Fatalities', weight="bold", size=20, transform=ax.transAxes)    
ax.set_xlabel('Months from present', fontsize=15)
ax.set_ylabel('Fatalities', fontsize=15)

ax.set_xlim(-36, 36)
ax.set_xticks([-36, -30, -33, -27,-24, -21, -18, -15, -12, -9, -6, -3, 0, 3, 6, 9,12, 15, 18, 21,24, 27,30, 33, 36])

ax.set_ylim(bottom=0)
ax.tick_params(axis='x', labelsize=12)
ax.tick_params(axis='y', labelsize=12)

for spine in ["top", "right", "bottom", "left"]:
    ax.spines[spine].set_visible(False)

plt.show()
